### Una vez extraido los directores de credits.csv podemos comenzar a hacer el ETL general. Haremos unos pequeños cambios para poder cumplir con los endpoints solicitados en nuestra api. tales como la modificacion de tipo de datos, el tratado de valores faltantes y quitar las columnas que no nos sirven.

In [1]:
#importamos las librerias necesarias.
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
#leemos el dataset con los directores incluidos.
df = pd.read_csv('data/movies_directores.csv', low_memory=False)
df.head()

,Unnamed: 0,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,directores
0,0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,['John Lasseter']
1,1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,['Joe Johnston']
2,2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,['Howard Deutch']
3,3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,...,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,['Forest Whitaker']
4,4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,...,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,['Charles Shyer']


In [3]:
df.shape

(45542, 26)

Okey nuestro dataset posee 26 columnas y 45542 peliculas, reduzcamos un poco estos valores dejando las columnas que necesitamos y eliminando duplicados.

In [4]:
df.columns

Index(['Unnamed: 0', 'adult', 'belongs_to_collection', 'budget', 'genres',
       'homepage', 'id', 'imdb_id', 'original_language', 'original_title',
       'overview', 'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'directores'],
      dtype='object')

In [5]:
df = df[['belongs_to_collection', 'budget', 'genres', 'id', 'original_language',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'title',
       'vote_average', 'vote_count', 'directores']]

In [6]:
df.drop_duplicates(subset='title', inplace=True)

In [7]:
df.shape

(42278, 17)

Observamos que nos quedaron 42278 peliculas y solo 17 columnas. ahora busquemos por valores faltantes.

In [8]:
df.isna().sum()

belongs_to_collection    37920
budget                       0
genres                       0
id                           0
original_language           11
overview                   915
popularity                   1
production_companies         1
production_countries         1
release_date                82
revenue                      1
runtime                    251
spoken_languages             1
title                        1
vote_average                 1
vote_count                   1
directores                   1
dtype: int64

Observamos varias columnas con muchos faltantes. 

Tiene sentido que "belongs_to_collection" posea tantos faltantes ya que las peliculas que pertenecen a una coleccion seran las unicas que tengan valor en ellas, podemos rellenar el faltante como que no tiene coleccion.

En otras columnas como release_date podriamos eliminar las filas con faltantes ya que no son muchas y pierden una informacion importante, al igual que con overview ya que nos sera util para hacer nuestro modelo de machine learning

Y por ultimo columnas como runtime podemos cambiar los nulos por el promedio de duracion de las peliculas para tener algo mas acertado.

In [9]:
df['belongs_to_collection'].fillna('no_collection', inplace=True)
df['original_language'].fillna('N/D', inplace=True)
df['overview'].dropna(inplace=True)
df['title'].dropna(inplace=True)
df['runtime'].fillna(df['runtime'].mean(), inplace=True)


In [10]:
df.dropna(subset='release_date', inplace=True)
#aprovechamos y cambiamos los tipos de datos 
df['release_date'] = pd.to_datetime(df['release_date'])
#y creamos una nueva columna con el año de estreno 
df['release_year'] = df['release_date'].dt.year


In [11]:
#se nos solicita crear una columna 'return' con el retorno de inversion de la pelicula,
# en esta celda trabajaremos en ello
df['budget'].fillna(0, inplace=True)
df['budget'] = df['budget'].astype('float32')
df['revenue'].fillna(0, inplace=True)
df['return'] = ((df['revenue'] - df['budget']) / df['budget']) * 100
df['return'].fillna(0, inplace=True)
df['return'].replace(np.inf, 0, inplace=True)
df['return'].head()

0    1145.180110
1     304.303460
2       0.000000
3     409.075975
4       0.000000
Name: return, dtype: float64

In [12]:
df.isna().sum()

belongs_to_collection      0
budget                     0
genres                     0
id                         0
original_language          0
overview                 902
popularity                 0
production_companies       0
production_countries       0
release_date               0
revenue                    0
runtime                    0
spoken_languages           0
title                      0
vote_average               0
vote_count                 0
directores                 1
release_year               0
return                     0
dtype: int64

Vemos que ya no tenemos valores faltantes y por lo tanto evitamos la mayor parte de los errores

In [13]:
df['directores'].dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 42196 entries, 0 to 45541
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  42196 non-null  object        
 1   budget                 42196 non-null  float32       
 2   genres                 42196 non-null  object        
 3   id                     42196 non-null  object        
 4   original_language      42196 non-null  object        
 5   overview               41294 non-null  object        
 6   popularity             42196 non-null  object        
 7   production_companies   42196 non-null  object        
 8   production_countries   42196 non-null  object        
 9   release_date           42196 non-null  datetime64[ns]
 10  revenue                42196 non-null  float64       
 11  runtime                42196 non-null  float64       
 12  spoken_languages       42196 non-null  object        
 13  title 

Tambien tenemos todas nuestras columnas con su respectivo tipo!

In [14]:
df.head(1)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,title,vote_average,vote_count,directores,release_year,return
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Toy Story,7.7,5415.0,['John Lasseter'],1995,1145.18011


Ahora tenemos que encargarnos de las columnas anidadas para extraer la informacion, lo haremos en 2 partes, comenzaremos con belongs_to_collection que tomo cada fila como un diccionario y luego el resto de columnas ya que son tomadas como fila.

In [15]:
#iniciamos lista vacia para las colecciones
collection = []
#iteramos en el dataframe
for i in range(0, df.shape[0]):
    #asignamos a row cada fila
    row = df.iloc[i,0]
    #si la lista tiene datos
    if row != 'no_collection':
        #convertimos en diccionario la fila
        row = eval(row)
        #agregamos el nombre de la coleccion a nuestra lista
        collection.append(row['name'])
    else:
        #sino, agregamos los datos vacios a la lista para no perder los indicies.
        collection.append(row)
# creamos un diccionario y lo convertimos en dataframe.
dicc = {'coleccion': collection}
df['belongs_to_collection'] = pd.DataFrame(dicc)
df.head(1)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,title,vote_average,vote_count,directores,release_year,return
0,Toy Story Collection,30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Toy Story,7.7,5415.0,['John Lasseter'],1995,1145.18011


Ahora creemos una funcion para el resto de columnas.

In [16]:
def desanidar_columnas(indice_columna, nombre_columna):
    """
    Esta funcion se utiliza para desanidar una columna
    que contenga una lista con diccionarios y dentro de esos
    diccionarios la key: 'name'. devolviendo un dataframe con
    los nombres.

    parametros:
    -------
    indice_columna: el indice de la columna del dataframe a desanidar.
    nombre_columna: el nombre de la columna que contiene los nombres en
    el dataframe de salida.
    """
    #creamos una lista vacia donde guardaremos los nombres
    columnas_names = []
    #Iteramos sobre el df
    for i in range(0, df.shape[0]):
        #instanciamos la fila:
        fila = df.iloc[i, indice_columna]
        #si la fila es nulo
        if fila == 'N/D':
            #aniadimos la fila y paramos el loop
            columnas_names.append(fila)
        #sino
        else:
            #convertimos de str a list
            fila = eval(fila)
            #guardaremos los nombres de este indice en otra sublista
            fila_names = []
            #iteramos la lista
            for i in fila:
                #agregamos los nombres
                fila_names.append(i['name'])
            #agregamos la lista de nombres de la fila a nuestra lista oficial.
            columnas_names.append(fila_names)

    #creamos el diccionario con los datos
    diccionario = {nombre_columna: columnas_names}
    #creamos y devolvemos el dataframe
    dataframe = pd.DataFrame(diccionario)
    return dataframe

In [17]:
#creamos los dataframes por columna
df_gen = desanidar_columnas(2, 'genres')
df_pc = desanidar_columnas(7, 'production_companies')
df_count = desanidar_columnas(8, 'production_countries')
df_spk = desanidar_columnas(12, 'spoken_languages')

#los reemplazamos en el datafram
df.genres = df_gen
df.production_companies = df_pc
df.production_countries = df_count
df.spoken_languages = df_spk
df.head(1)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,title,vote_average,vote_count,directores,release_year,return
0,Toy Story Collection,30000000.0,"[Animation, Comedy, Family]",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,[Pixar Animation Studios],[United States of America],1995-10-30,373554033.0,81.0,[English],Toy Story,7.7,5415.0,['John Lasseter'],1995,1145.18011


In [18]:
df.spoken_languages = df.spoken_languages.astype('object')
df.spoken_languages.fillna('N/D', inplace=True)
df.spoken_languages.value_counts()

df.production_countries = df.production_countries.astype('object')
df.production_countries.fillna('N/D', inplace=True)
df.production_countries.value_counts()

df.production_companies = df.production_companies.astype('object')
df.production_companies.fillna('N/D', inplace=True)
df.production_companies.value_counts()


production_companies
[]                                                                                                                                      10120
N/D                                                                                                                                      3026
[Metro-Goldwyn-Mayer (MGM)]                                                                                                               644
[Warner Bros.]                                                                                                                            484
[Paramount Pictures]                                                                                                                      447
                                                                                                                                        ...  
[Imagine Entertainment, Malpaso Productions, Relativity Media]                                                                 

In [19]:
df.directores = df.directores.astype('object')
df.directores.fillna('N/D', inplace=True)

Ahora ya estamos listos para hacer el EDA y comenzar con los endpoints, acompañanos con ello.

In [20]:
# extraemos el dataset ya listo.
df.to_csv('data/movies_etl.csv')